In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import mdtraj as md
import mcg_waters as mcg
import h5py
import pandas as pd
import matplotlib_config as cfg
import fnmatch
from matplotlib import cm

In [2]:
mu = 170
run = 0
idx = 2
stride = 10
burst = 1e5

In [5]:
folder = f"data_mu{mu}_0/run_{run}/"

conffolder = "gro_files"

In [6]:
# Input files
dcd_file = os.path.join(folder, f"traj_{idx}.dcd")
topology_file = os.path.join(conffolder, "conf.gro")


# Load the trajectory with a stride of n (e.g., every 10th frame)
n = stride  # Change this to the desired interval
traj = md.load(dcd_file, top=topology_file, stride=n)
original_frame_indices = np.arange(traj.n_frames) * n

traj.time = np.arange(0, traj.n_frames) * burst * n

print(f"Trajectory loaded using every {n}th step")


Trajectory loaded using every 10th step


In [7]:
times_ps, CAR_COM_frames, WATER_COM_frames, box_lengths = mcg.COM_calculation_mdtraj(traj)
total_frames = len(times_ps)

MCG_OP_new_results = []
monomer_coords_all_frames = []
water_coords_all_frames = []


for idx, time_ps in enumerate(times_ps):
    CAR_COM = CAR_COM_frames[idx]
    WATER_COM = WATER_COM_frames[idx]
    box_length = box_lengths[idx]
    
    # Assuming `MCG` takes these arguments and computes the order parameter
    MCG_OP, _, monomer_coords, water_coords = mcg.MCG_optimized(CAR_COM, WATER_COM, box_length, guest_cutoff=0.9)
    print(MCG_OP)
    print(len(monomer_coords))
    #np.savetxt( '../mcg_tests/monomers.xyz', monomer_coords)
    #print(f'\rProcessing frame {idx + 1}/{total_frames}', end='', flush=True)
    #print(f'The largest cluster size in the system at frame {frame} and time {round(time_ps, 10)} is: {MCG_OP}')
    MCG_OP_new_results.append(MCG_OP)
    monomer_coords_all_frames.append(monomer_coords)
    water_coords_all_frames.append(water_coords)



10
10
16
16
53
53
52
52
44
44
37
37
53
53
72
72
64
64
74
74
58
58
72
72
57
57
76
76
80
80
87
87
78
78
86
86
95
95
105
105
122
122
144
144
149
149
140
140
147
147
145
145
168
168
174
174
174
174
164
164
162
162
172
172
168
168
181
181
179
179
195
195
186
186
232
232
206
206
188
188
216
216
190
190
181
181
178
178
198
198
200
200
193
193
190
190
183
183
187
187
205
205
199
199
215
215
207
207
210
210
220
220
221
221
230
230
247
247
222
222
232
232
251
251
225
225
237
237
237
237
230
230
259
259
252
252
245
245
266
266
272
272
266
266
259
259
269
269
273
273
276
276
257
257
266
266
272
272
295
295
281
281
283
283
276
276
290
290
292
292
321
321
328
328
304
304
319
319
314
314
331
331
329
329
331
331
329
329
337
337
325
325
342
342
355
355
332
332
342
342
340
340
367
367
368
368
378
378
345
345
362
362
361
361
363
363
389
389
390
390
389
389
390
390
392
392
380
380
372
372
397
397
389
389
398
398
417
417
412
412
425
425
423
423
436
436
436
436
415
415
423
423
440
440
442
442
448
448
451
45

In [8]:
print(box_lengths)

[4.839941, 4.828114, 4.835199, 4.840999, 4.8339896, 4.8289056, 4.8281665, 4.8358746, 4.828008, 4.8367124, 4.8308024, 4.8387723, 4.8474855, 4.8370857, 4.8297105, 4.828304, 4.8319516, 4.829252, 4.827729, 4.830056, 4.8218455, 4.8242774, 4.8184266, 4.827148, 4.8222933, 4.8246455, 4.827661, 4.815333, 4.821996, 4.8108225, 4.8310337, 4.822435, 4.814834, 4.814178, 4.8139367, 4.815571, 4.8217444, 4.8217034, 4.8133197, 4.80766, 4.8129115, 4.812463, 4.8106217, 4.8182354, 4.813176, 4.8190484, 4.8206177, 4.8219666, 4.8256454, 4.81558, 4.8144455, 4.8134913, 4.809939, 4.8174906, 4.812865, 4.8110995, 4.823163, 4.81572, 4.8041344, 4.8111553, 4.8104305, 4.8165917, 4.803033, 4.8059387, 4.811128, 4.8171353, 4.810118, 4.8077407, 4.8156943, 4.81195, 4.81031, 4.8066974, 4.8223476, 4.8114543, 4.812264, 4.810067, 4.813977, 4.8235316, 4.8109927, 4.806194, 4.8035774, 4.8017917, 4.8034472, 4.8046002, 4.8200545, 4.805465, 4.805549, 4.8066707, 4.8058696, 4.8055325, 4.802663, 4.8137345, 4.810791, 4.8039875, 4.814529

In [9]:
def write_gro(filename, coords_frames, times_ps, resname="MON", atomname="C", box_lengths = box_lengths):
    with open(filename, 'w') as f:
        for idx, (coords, time_ps) in enumerate(zip(coords_frames, times_ps)):
            num_atoms = coords.shape[0]
            f.write(f"Generated frame {idx}, Time: {time_ps:.2f} ps\n")
            f.write(f"{num_atoms}\n")
            for i, (x, y, z) in enumerate(coords * 10):  # Convert nm → Å
                res_id = 1
                atom_id = i + 1
                # Write GRO line with fixed-width format
                f.write(f"{res_id:5d}{resname:<5}{atomname:<5}{atom_id:5d}"
                        f"{x/10:8.3f}{y/10:8.3f}{z/10:8.3f}\n")  # Convert Å back to nm
            # Box size line (assuming cubic box)
            f.write(f"{box_lengths[idx]:.5f} {box_lengths[idx]:.5f} {box_lengths[idx]:.5f}\n")


In [10]:
write_gro("monomers.gro", monomer_coords_all_frames, times_ps)


In [11]:
output_xyz_path = "monomers.xyz"

with open(output_xyz_path, 'w') as f:
    for idx, (time_ps, monomer_coords) in enumerate(zip(times_ps, monomer_coords_all_frames)):
        print(idx)
        num_atoms = monomer_coords.shape[0]
        f.write(f"{num_atoms}\n")
        f.write(f"Frame {idx}, Time: {time_ps:.2f} ps\n")
        for atom_pos in monomer_coords:
            x, y, z = atom_pos * 10
            f.write(f"C {x:.6f} {y:.6f} {z:.6f}\n")  # "C" as placeholder atom type


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137


In [12]:
traj.save("170_run0_idx2.gro")   

In [13]:
for i in range(len(water_coords_all_frames)):
    positions = np.array(water_coords_all_frames[i])
    print(np.shape(positions))

(62, 5, 3)
(80, 5, 3)
(137, 5, 3)
(115, 5, 3)
(112, 5, 3)
(120, 5, 3)
(169, 5, 3)
(163, 5, 3)
(137, 5, 3)
(164, 5, 3)
(143, 5, 3)
(164, 5, 3)
(150, 5, 3)
(180, 5, 3)
(199, 5, 3)
(185, 5, 3)
(161, 5, 3)
(200, 5, 3)
(190, 5, 3)
(226, 5, 3)
(254, 5, 3)
(285, 5, 3)
(322, 5, 3)
(271, 5, 3)
(307, 5, 3)
(304, 5, 3)
(344, 5, 3)
(356, 5, 3)
(376, 5, 3)
(338, 5, 3)
(331, 5, 3)
(319, 5, 3)
(332, 5, 3)
(378, 5, 3)
(359, 5, 3)
(403, 5, 3)
(396, 5, 3)
(450, 5, 3)
(360, 5, 3)
(388, 5, 3)
(409, 5, 3)
(358, 5, 3)
(362, 5, 3)
(364, 5, 3)
(431, 5, 3)
(400, 5, 3)
(386, 5, 3)
(415, 5, 3)
(375, 5, 3)
(395, 5, 3)
(428, 5, 3)
(421, 5, 3)
(412, 5, 3)
(409, 5, 3)
(408, 5, 3)
(459, 5, 3)
(450, 5, 3)
(475, 5, 3)
(492, 5, 3)
(488, 5, 3)
(505, 5, 3)
(513, 5, 3)
(492, 5, 3)
(527, 5, 3)
(506, 5, 3)
(509, 5, 3)
(526, 5, 3)
(536, 5, 3)
(540, 5, 3)
(549, 5, 3)
(570, 5, 3)
(540, 5, 3)
(536, 5, 3)
(602, 5, 3)
(566, 5, 3)
(575, 5, 3)
(563, 5, 3)
(586, 5, 3)
(586, 5, 3)
(634, 5, 3)
(574, 5, 3)
(575, 5, 3)
(595, 5, 3)
(670, 

In [14]:
output_xyz_path = "waters.xyz"

with open(output_xyz_path, 'w') as f:
    for idx, (time_ps, waters_in_frame) in enumerate(zip(times_ps, water_coords_all_frames)):
        print(idx)
        waters_in_frame = np.array(waters_in_frame)
        if len(waters_in_frame) == 0:
            continue  # skip if no relevant waters in this frame

        # Reshape to (N*5, 3) for XYZ format
        all_coords = waters_in_frame.reshape(-1, 3)  # flatten to (num_atoms, 3)
        num_atoms = all_coords.shape[0]
        
        f.write(f"{num_atoms}\n")
        f.write(f"Frame {idx}, Time: {time_ps:.2f} ps\n")
        
        for atom_pos in all_coords:
            x, y, z = atom_pos * 10  # Convert nm to Å
            f.write(f"O {x:.6f} {y:.6f} {z:.6f}\n")  # Change "O" if needed


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137


In [15]:
62*5

310